<a href="https://www.ired.info/"> <img src="file/ired_logo.png" alt="ired" style="width: 150px;"/> </a>

# Object Classification - Data Collection 物件分類 - 收集數據

### Display PI camera 顯示鏡頭

First, initialize and display the PI camera.

The neural network takes a 224x224 pixel image as input, which is same for Teachable Machine
<br>The camera is set to 224x224 to reduce the collected image size and fit into neural work.

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

### Create Interactive Button for Data Collection 創健收集數據按鈕

New folders are created to store collected data. 

A folder ``dataset``  will contain two sub-folders ``press`` 按下 and ``stop`` 停止, 
where each folder consider as a class in classification

In [ ]:
import os

press_dir = 'dataset/press'
stop_dir = 'dataset/stop'

try:
    os.makedirs(press_dir)
    os.makedirs(stop_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

If you press the refresh button on the left, you should now see new dataset folder created, with press and stop folder inside dataset folder.

To capture image, you will create 2 button for each class.
<br>A text box will also know how many images you have already captured.

In [ ]:
button_layout = widgets.Layout(width='128px', height='64px')
stop_button = widgets.Button(description='add stop', button_style='danger', layout=button_layout)
press_button = widgets.Button(description='add press', button_style='success', layout=button_layout)
stop_count = widgets.IntText(layout=button_layout, value=len(os.listdir(stop_dir)))
press_count = widgets.IntText(layout=button_layout, value=len(os.listdir(press_dir)))

display(widgets.HBox([stop_count, stop_button]))
display(widgets.HBox([press_count, press_button]))

The buttons display cannot add any image yet.
<br> You have to attach functions to save images for each class to the buttons' ``on_click`` event.

To make sure image file names are not repeated , the ``uuid`` package in python is used. 
<br> It is a unique identifer generated from current time and machine address.

In [ ]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_stop():
    global stop_dir, stop_count
    save_snapshot(stop_dir)
    stop_count.value = len(os.listdir(stop_dir))
    
def save_press():
    global press_dir, press_count
    save_snapshot(press_dir)
    press_count.value = len(os.listdir(press_dir))
    
stop_button.on_click(lambda x: save_stop())
press_button.on_click(lambda x: save_press())

Now the buttons above should work and save images to the ``stop`` and ``press`` directories.  You can view the images by clicking them

Now go ahead and collect some data (**stop: 閒置狀態, press: 按下狀態**)

1. Place the camera in a scenario where hand sanitizer is pressed and click on button ``add press``
2. Place the camera in a scenario where hand sanitizer is stopped and click on button ``add stop``
mw
> Useful TIPS: Next code will display interactive button and camera together to make data collection easier

Here are some general tips for labeling data, to improve training model.
<br> 收集數據的小技巧:
1. Different orientations 不同方位
2. Different lighting 不同燈光
3. Varied background 不同背景

The more images captured with different scenarios, the machine is accurate in most position in real world.
<br> It is important to get varied data but not a lot of similar data, you will  need at least 50 images of each class.

### Put Interactive button and camera together 將收集數據按鈕與鏡頭並排

In [ ]:
display(image)
display(widgets.HBox([stop_count, stop_button, press_count, press_button]))

After you collect at least **50 images of each class (閒置及按下各50張)**, stop the camera and go to the next Jupyter Notebook to start training a model on Jetson Nano.

### Stop Camera 關閉鏡頭
Properly stop the camera.

In [ ]:
camera.stop()

---End---